In [4]:
import pandas as pd

def csv_to_las_custom(csv_file, las_file, well_name="Well_1"):
    # Read the CSV
    df = pd.read_csv(csv_file)

    # Move DEPT to the first column if not already
    cols = list(df.columns)
    if "DEPT" not in cols:
        raise ValueError("CSV must have a 'DEPT' column for depth.")
    cols.remove("DEPT")
    cols.insert(0, "DEPT")
    df = df[cols]

    # Calculate start, stop, step
    start_depth = df["DEPT"].iloc[0]
    stop_depth = df["DEPT"].iloc[-1]
    step = df["DEPT"].iloc[1] - df["DEPT"].iloc[0]

    # Build LAS sections
    las_lines = []

    # ~Version
    las_lines.append("~Version Information")
    las_lines.append(" VERS. 2.0 : LAS version")
    las_lines.append(" WRAP. NO  : No wrap")

    # ~Well
    las_lines.append("~Well Information")
    las_lines.append(f" WELL. {well_name} : Well name")
    las_lines.append(f" STRT. {start_depth:.4f} : Start depth")
    las_lines.append(f" STOP. {stop_depth:.4f} : Stop depth")
    las_lines.append(f" STEP. {step:.4f} : Step size")
    las_lines.append(" NULL. -999.25 : Null value")

    # ~Curve
    las_lines.append("~Curve Information")
    for col in df.columns:
        las_lines.append(f" {col:<10}. : {col}")

    # ~Parameter
    las_lines.append("~Parameter Information")
    las_lines.append(" RUN. Unknown : Run number")

    # ~ASCII
    las_lines.append("~ASCII Log Data")
    for _, row in df.iterrows():
        line = "  ".join(f"{val:10.4f}" if pd.notnull(val) else "  -999.25" for val in row)
        las_lines.append(line)

    # Save to LAS
    with open(las_file, "w") as f:
        for line in las_lines:
            f.write(line + "\n")

    print(f"LAS file written to: {las_file}")


In [5]:
csv_to_las_custom("Hikmat-01.csv", "HIKMAT-1_MM_15.las", well_name="HIKMAT-1")


LAS file written to: HIKMAT-1_MM_15.las
